In [1]:
import pandas as pd 

data = pd.read_csv("broker.csv").replace('-', '', regex=True)
data["Größe in m² (Kataster)"] = pd.to_numeric(data["Größe in m² (Kataster)"], downcast="float")

#data.head()

In [2]:
totalGreens = data['Größe in m² (Kataster)'].sum()
print("%s square metre" % totalGreens)
print("%s hectare" % (totalGreens/10000))

61376304.0 square metre
6137.6304 hectare


In [3]:
#len(data['Nummer des Planungsraumes'].unique().tolist())

In [4]:
#list(filter(None, data['Name Planungsraum'].unique().tolist()))

In [5]:
#data.columns

In [6]:
#data_open_spaces = data.groupby('Nummer des Planungsraumes', as_index=False).agg({"Größe in m² (Kataster)": "sum"})
data_open_spaces = data_open_spaces.drop([0, 411])
data_open_spaces

In [7]:
#data.groupby('Name Planungsraum', as_index=False).size().reset_index(name='Anzahl der Grünflächen').sort_values(['Anzahl der Grünflächen'], ascending=False).style.hide_index()


In [8]:
#data.groupby('Bezirk').size().reset_index(name='Anzahl der Grünflächen').sort_values(['Anzahl der Grünflächen'], ascending=False).style.hide_index()

In [9]:
#empty_PLR = data["Name Planungsraum"] == ""
#data[empty_PLR]